In [1]:
from pprint import pprint
import numpy as np
import graphviz
import gtsam
import argparse
import open3d as o3d
from scipy.spatial.transform import Rotation as R
from gtsam import (
    Pose3,
    Rot3,
    Point3,
    Values,
    NonlinearFactorGraph,
    PriorFactorPose3,
    BetweenFactorPose3,
    Symbol,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dir(gtsam.GncLMOptimizer)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'getInlierCostThresholds',
 'getWeights',
 'optimize',
 'setInlierCostThresholds',
 'setInlierCostThresholdsAtProbability',
 'setWeights']

In [3]:
dir(Symbol)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'chr',
 'equals',
 'index',
 'key',
 'print',
 'string']

In [4]:
# Qx Qy Qz Qw
R.from_quat((0, 0, 0, 1)).as_matrix()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [5]:
a = np.array([[1, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1], [0, 0, 0, 1]])
print(a[:3, :3])

[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [6]:
# GNC LM Optimizer
LM_params = gtsam.LevenbergMarquardtParams()
gnc_lm_params = gtsam.GncLMParams(LM_params)

print("Welcome to RRC DPGO")

Robot_Symbol = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]

Link1_6dof = gtsam.noiseModel.Diagonal.Sigmas([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
Link2_6dof = gtsam.noiseModel.Diagonal.Sigmas([0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
Prior_6dof = gtsam.noiseModel.Diagonal.Sigmas([0, 0, 0, 0, 0, 0])

Welcome to RRC DPGO


In [7]:
from gtsam import *

# ... (Previous code)

# Create factor graph and values
g = NonlinearFactorGraph()
v = Values()

# Define keys for the robots
key1 = Symbol("B", 1).key()
key2 = Symbol("B", 2).key()

# Create a BetweenFactorPose3 and add it to the factor graph
relative_pose = Pose3()  # Adjust the relative pose as needed
factor = BetweenFactorPose3(key1, key2, relative_pose, Prior_6dof)
g.add(factor)

# Print the factor graph
g.print()

NonlinearFactorGraph: size: 1

Factor 0: BetweenFactor(B1,B2)
  measured:  R: [
	1, 0, 0;
	0, 1, 0;
	0, 0, 1
]
t: 0 0 0
  noise model: constrained sigmas [0; 0; 0; 0; 0; 0];
  noise model: constrained mu [1000; 1000; 1000; 1000; 1000; 1000];



In [8]:
class Robot:
    def __init__(self, robot_id, file_path):
        self.robot_id = robot_id
        self.file_path = file_path
        self.poses = []
        self.edges = []
        self.symbol = Robot_Symbol[robot_id]
        self.read_g2o_file()

    # getters and setters
    def set_symbol(self, symbol):
        self.symbol = symbol

    def set_path(self, file_path):
        self.file_path = file_path

    def set_poses(self, poses):
        self.poses = poses

    def set_edges(self, edges):
        self.edges = edges

    def set_robot_id(self, robot_id):
        self.robot_id = robot_id

    def get_file_path(self):
        return self.file_path

    def get_poses(self):
        return self.poses

    def get_edges(self):
        return self.edges

    def get_robot_id(self):
        return self.robot_id

    def get_symbol(self):
        return self.symbol

    def read_g2o_file(self):
        with open(self.file_path, "r") as f:
            for line in f:
                data = line.strip().split()
                if len(data) == 0:
                    continue
                if data[0] == "VERTEX_SE3:QUAT":  # Check if it's an SE3 vertex line
                    pose_id = int(data[1])
                    # pose_id = self.get_symbol()+str(pose_id)
                    x, y, z = float(data[2]), float(data[3]), float(data[4])
                    qw, qx, qy, qz = (
                        float(data[5]),
                        float(data[6]),
                        float(data[7]),
                        float(data[8]),
                    )

                    pose = Pose3(Rot3.Quaternion(
                        qw, qx, qy, qz), Point3(x, y, z))

                    self.poses.append((pose_id, pose))

                elif data[0] == "EDGE_SE3:QUAT":  # Check if it's an SE3 edge line
                    # print(data, len(data))
                    # edge_id = int(data[1])
                    from_node_id = int(data[1])
                    to_node_id = int(data[2])
                    x, y, z = float(data[3]), float(data[4]), float(data[5])
                    qx, qy, qz, qw = (
                        float(data[6]),
                        float(data[7]),
                        float(data[8]),
                        float(data[9]),
                    )

                    # Normalize the quaternion
                    normalized_quaternion = (qx, qy, qz, qw) / np.linalg.norm(
                        (qx, qy, qz, qw)
                    )

                    rotation_matrix = R.from_quat(normalized_quaternion)

                    rotation_matrix = np.array(rotation_matrix.as_matrix())

                    # print( from_node_id, to_node_id, x, y, z,qx,qy,qz,qw, '\n')
                    # print(rotation_matrix.ravel())
                    assert np.linalg.det(rotation_matrix)

                    edge_pose = Pose3(
                        Rot3(rotation_matrix.ravel()), Point3(x, y, z))

                    self.edges.append((from_node_id, to_node_id, edge_pose))      

In [9]:
path1 = "./Traj_Data/traj.txt"
path2 = "./Traj_Data/traj.txt"

# Read the data
robot1 = Robot(1, path1)
print(robot1.poses)
# print(robot1.poses)
# print(robot1.edges[0][0])

robot2 = Robot(2, path1)

[(0, R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 0 0 0
), (1, R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 1 0 0
), (2, R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 2 0 0
), (3, R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 3 0 0
)]


In [10]:
class Submap:
    def __init__(self, submap_id):
        self.submap_id = submap_id

        self.robots = []
        self.nodes = []
        self.edges = []

        # self.graph_viz = Digraph(comment="Submap Graph {}".format(submap_id))
        self.graphViz = graphviz.Graph()
        self.graph = NonlinearFactorGraph()

        self.initial = Values()
        self.optimized = Values()

        self.graph.add(PriorFactorPose3(
            0, Pose3(Rot3(), Point3(0, 0, 0)), Prior_6dof))
        self.initial.insert(0, Pose3())

    def add_robot(self, robot):
        self.robots.append(robot)
        for i in range(len(robot.poses)):
            self.add_node(
                Symbol(robot.get_symbol(), int(
                    robot.poses[i][0])), robot.poses[i][1]
            )
        for i in range(len(robot.edges)):
            self.add_edge(
                Symbol(robot.get_symbol(), int(robot.edges[i][0])),
                Symbol(robot.get_symbol(), int(robot.edges[i][1])),
                robot.edges[i][2],
            )

    def add_node(self, node_id, pose):
        self.nodes.append((node_id, pose))
        # print(node_id, pose)
        self.initial.insert(node_id.key(), pose)
        # self.graphViz.node(str(node_id.key()))

    def add_edge(self, from_node_id, to_node_id, edge_matrix):
        self.edges.append((from_node_id, to_node_id, edge_matrix))
        self.graph.add(
            BetweenFactorPose3(
                from_node_id.key(), to_node_id.key(), edge_matrix, Link1_6dof
            )
        )
        self.graphViz.edge(str(from_node_id), str(to_node_id))

    def print_submap(
        self, view_Edge=True, view_Nodes=True, view_Graph=False, view_Initial=False, save_Graph=False, graph_path='./DPGO_graph'
    ):
        print(f"Submap ID: {self.submap_id}")
        print("\nRobots:")
        print(self.robots)
        if view_Nodes == True:
            print("\nVertices:\n")
            for pose in self.nodes:
                print(pose)
        if view_Edge == True:
            print("\nEdges:\n")
            for edge in self.edges:
                print(edge)
        if view_Initial == True:
            self.initial.print("\n Values : \n")
        if view_Graph == True:
            self.graph.print("\nGraph: \n")
        if save_Graph:
            self.graphViz.render(graph_path, view=False)

    def optimize(self):
        optimizer = gtsam.GncLMOptimizer(
            self.graph, self.initial, gnc_lm_params)
        self.optimized = optimizer.optimize()
        print(self.optimized)
        print(optimizer.getWeights())
        print(optimizer.getInlierCostThresholds())
        pass

In [11]:
submap1 = Submap(1)
# print(submap1.graph)
submap1.add_robot(robot1)
submap1.add_robot(robot2)
# submap1.add_edge()
# submap1.print_submap(view_Edge=False, view_Graph=True, view_Initial=False, view_Nodes=False, save_Graph=True)
submap1.add_edge(Symbol("B", 1), Symbol("C", 1),
                 Pose3(Rot3(), Point3(0, 0, 0)))
# submap1.print_submap(view_Edge=False, view_Graph=True, view_Initial=False, view_Nodes=False, save_Graph=True, graph_path='./DPGO_graph2')
submap1.optimize()

Values with 9 values:
Value 0: (gtsam::Pose3)
R: [
	1, 0, 0;
	0, 1, 0;
	0, 0, 1
]
t: 0 0 0

Value B0: (gtsam::Pose3)
R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 0 0 0

Value B1: (gtsam::Pose3)
R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 1 0 0

Value B2: (gtsam::Pose3)
R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 2 0 0

Value B3: (gtsam::Pose3)
R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 3 0 0

Value C0: (gtsam::Pose3)
R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 0 0 0

Value C1: (gtsam::Pose3)
R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 1 0 0

Value C2: (gtsam::Pose3)
R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 2 0 0

Value C3: (gtsam::Pose3)
R: [
	-1, 0, 0;
	0, -1, 0;
	0, 0, 1
]
t: 3 0 0


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[8.40594691 8.40594691 8.40594691 8.40594691 8.40594691 8.40594691
 8.40594691 8.40594691 8.40594691 8.40594691 8.40594691 8.40594691]


In [13]:
class Map:
    def __init__(self):
        self.submaps = {}

    def add_submap(self, submap_id):
        if submap_id not in self.submaps:
            self.submaps[submap_id] = Submap(submap_id)
        else:
            print(f"Submap {submap_id} already exists")

    def add_robot_to_submap(self, submap_id, robot):
        if submap_id not in self.submaps:
            print(f"Submap {submap_id} does not exist")
            return
        self.submaps[submap_id].add_robot(robot)

    # not required
    def add_pose_to_submap(self, submap_id, pose):
        if submap_id not in self.submaps:
            print(f"Submap {submap_id} does not exist")
            return
        self.submaps[submap_id].add_node(pose)

    # # merge map based on the different cliques found using connected sets
    def add_edge_to_submap(self, submap_id, from_node_id, to_node_id, edge_pose3):
        if submap_id not in self.submaps:
            print(f"Submap {submap_id} does not exist")
            return
        self.submaps[submap_id].add_edge(from_node_id, to_node_id, edge_pose3)
    
    
    # def add_loop_to_submap(self, submap_id, from_node_id, to_node_id, edge_pose3):
    #     if submap_id not in self.submaps:
    #         print(f"Submap {submap_id} does not exist")
    #         return
    #     self.submaps[submap_id].add_edge(from_node_id, to_node_id, edge_pose3)

    # rewrtie this function
    def merge_submaps(self, robot1_id, robot2_id):
        submap1_id = f"submap_{robot1_id}"
        submap2_id = f"submap_{robot2_id}"

        # If submaps do not exist, create them
        if submap1_id not in self.submaps:
            self.add_submap(submap1_id)
        if submap2_id not in self.submaps:
            self.add_submap(submap2_id)

        submap1 = self.submaps[submap1_id]
        submap2 = self.submaps[submap2_id]

        combined_submap_id = f"submap_{robot1_id}_{robot2_id}"
        combined_submap = Submap(combined_submap_id)

        # Merge poses
        combined_submap.nodes = submap1.nodes + submap2.nodes

        # Merge edges
        combined_submap.edges = submap1.edges + submap2.edges

        # Add robots to the combined submap
        combined_submap.robots = list(set(submap1.robots + submap2.robots))

        # Remove individual submaps for the robots
        del self.submaps[submap1_id]
        del self.submaps[submap2_id]

        # Add the combined submap to the map
        self.submaps[combined_submap_id] = combined_submap

    def print_map(self):
        print("Map:")
        for submap_id, submap in self.submaps.items():
            submap.print_submap()
    
    def loop_closure_detector():
        pass

In [ ]:
def main(args):
    map_obj = Map()
    Robots = []
    for i in range(args.num_files):
        file_name = args.file_names[i]
        print(f"Processing file {i+1}: {file_name}")
        robot = Robot(i, file_name)
        robot.read_g2o_file()
        Robots.append(robot)

    for robot in Robots:
        # After reading all the files, add poses and edges to the map
        submap_id = f"submap_{robot.robot_id}"
        map_obj.add_submap(submap_id)
        map_obj.add_robot_to_submap(submap_id, robot.robot_id)
        for pose in robot.poses:
            map_obj.add_pose_to_submap(submap_id, pose)
        for from_node_id, to_node_id, edge_pose in robot.edges:
            map_obj.add_edge_to_submap(submap_id, from_node_id, to_node_id, edge_pose)

    # After reading all the files and creating submaps for each robot, check for loop closures and merge submaps if needed
    # For example, assuming loop closures between robot 0 and robot 1:
    robot1_id = 0
    robot2_id = 1
    map_obj.merge_submaps(robot1_id, robot2_id)

    # Now you can access the combined submap for robots 0 and 1 by using their combined submap id, e.g., "submap_0_1"
    combined_submap_id = f"submap_{robot1_id}_{robot2_id}"
    combined_submap = map_obj.submaps[combined_submap_id]
    print(f"Combined Submap Poses: {combined_submap.nodes}")
    print(f"Combined Submap Edges: {combined_submap.edges}")

    # Print the entire map with all submaps, vertices, and edges
    map_obj.print_map()


class Args:
    pass


if __name__ == "__main__":
    # parser = argparse.ArgumentParser(description="Process multiple files")
    # # Add arguments to the parser
    # parser.add_argument("num_files", type=int, help="Number of files to be processed")
    # parser.add_argument(
    #     "file_names", nargs="+", help="Names of the input files (separated by space)"
    # )

    # # Parse the command-line arguments
    # args = parser.parse_args()
    # args = {num_files: 2, file_names: ["../data/robot0.g2o", "../data/robot1.g2o"]}
    args = Args()
    args.num_files = 2
    args.file_names = ["./Traj_Data/traj.txt", "./Traj_Data/traj.txt"]
    # Call the main function with the parsed arguments
    main(args)

Processing file 1: ./Traj_Data/traj.txt
Processing file 2: ./Traj_Data/traj.txt


AttributeError: 'int' object has no attribute 'poses'